참고
- `@trace_function()` 은 제가 자체적으로 작성한 데코레이터입니다. 경로는 `utils/util.py` 입니다.

- 랭그래프를 실행하기 위한 라이브러리 목록은 `modules/base.py` 에 있습니다.

In [11]:
from modules.base import *

#### 메시지 이해하기
- 노드는 특정 기능을 수행하는 함수

- 에지는 노드를 연결하는 역할

---

##### 1. override : state는 기본적으로 override 됨.

In [2]:
class TypedDictState(TypedDict):
    cnt: int
    
@trace_function()
def node_1(state):
    return {"cnt": state['cnt'] + 1}

builder = StateGraph(TypedDictState)
builder.add_node("node_1", node_1)
builder.add_edge(START, "node_1")
builder.add_edge("node_1", END)
graph = builder.compile()
graph.invoke({"cnt":1})


🚀 Passing Through [node_1] ..

#### [Input State]
  args: ({'cnt': 1},)
  kwargs: {}

#### [Output State]
  result: {'cnt': 2}


{'cnt': 2}

##### 2. append : append 되는 형태로 하면, 멀티턴 쉽게 구현 가능

- 단순 state 일 경우 `from operator import add` 사용

In [3]:
class State(TypedDict): 
    cnt: Annotated[list[int], add]

@trace_function()
def node_1(state):
    return {"cnt": [state['cnt'][0] + 1]}

builder = StateGraph(State)
builder.add_node("node_1", node_1)
builder.add_edge(START, "node_1")
builder.add_edge("node_1", END)
graph = builder.compile()
graph.invoke({"cnt":[1]})


🚀 Passing Through [node_1] ..

#### [Input State]
  args: ({'cnt': [1]},)
  kwargs: {}

#### [Output State]
  result: {'cnt': [2]}


{'cnt': [1, 2]}

- messages 일 경우 `from langgraph.graph.message import add_messages` 사용

- 중요한점은, add_messages를 추가한 MessagesState 클래스를 직접 선언해도되지만

- 사전에 정의된 `from langgraph.graph import MessagesState` 를 사용해도됨

In [4]:
# option 1 : 직접 정의
class MessagesState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

# option 2 : 사전에 정의된 Component
# from langgraph.graph import MessagesState

# option 3 : 상속받기 (아래처럼 상속받으면, messages 와 summary 사용 가능)
# class State(MessagesState):
#     summary: str

@trace_function()
def node_1(state:MessagesState):
    return {"messages": [llm.invoke(state["messages"])]}

@trace_function()
def node_2(state:MessagesState):
    return {"messages": [llm.invoke(state["messages"] + [HumanMessage(content="what is my name?")])]}

# Build graph
builder = StateGraph(MessagesState)
builder.add_node("node_1", node_1)
builder.add_node("node_2", node_2)
builder.add_edge(START, "node_1")
builder.add_edge("node_1", "node_2")
builder.add_edge("node_2", END)
graph = builder.compile()
result = graph.invoke({"messages":"Hi, this is changwoo."})


🚀 Passing Through [node_1] ..

#### [Input State]
  args: ({'messages': [HumanMessage(content='Hi, this is changwoo.', additional_kwargs={}, response_metadata={}, id='bfaca658-a2ef-440a-9265-9453f7e7b391')]},)
  kwargs: {}

#### [Output State]
  result: {'messages': [AIMessage(content='Hello Changwoo! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 14, 'total_tokens': 26, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_703d4ff298', 'finish_reason': 'stop', 'logprobs': None}, id='run-6b66d71b-ae61-4218-882a-070c92fbca8d-0', usage_metadata={'input_tokens': 14, 'output_tokens': 12, 'total_tokens': 26, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': 

##### 3. add_messages 이해
- id 값이 없을 경우 무조건 append
- id 값이 있을 경우, 해당 id 값이 있는 경우 overwrite

In [5]:
initial_messages = [AIMessage(content="Hello! How can I assist you?", 
                              name="Model"),
                    HumanMessage(content="I'm looking for information on marine biology.", 
                                 name="Lance")]
new_message = AIMessage(content="Sure, I can help with that. What specifically are you interested in?", 
                        name="Model")
add_messages(initial_messages , new_message)

[AIMessage(content='Hello! How can I assist you?', additional_kwargs={}, response_metadata={}, name='Model', id='208c9916-f5e6-4055-b7ff-92aef5be4e4f'),
 HumanMessage(content="I'm looking for information on marine biology.", additional_kwargs={}, response_metadata={}, name='Lance', id='51f15061-e8e8-40d1-be27-975489f805f4'),
 AIMessage(content='Sure, I can help with that. What specifically are you interested in?', additional_kwargs={}, response_metadata={}, name='Model', id='b729b556-6fb2-4fe0-b5cc-845757933e56')]

In [6]:
# with id -> append and overwrite
initial_messages = [AIMessage(content="Hello! How can I assist you?", 
                              name="Model", 
                              id="1"),
                    HumanMessage(content="I'm looking for information on marine biology.", 
                                 name="Lance", 
                                 id="2")]
new_message = HumanMessage(content="I'm looking for information on whales, specifically", 
                           name="changwoo", 
                           id="2")
add_messages(initial_messages , new_message)

[AIMessage(content='Hello! How can I assist you?', additional_kwargs={}, response_metadata={}, name='Model', id='1'),
 HumanMessage(content="I'm looking for information on whales, specifically", additional_kwargs={}, response_metadata={}, name='changwoo', id='2')]

##### 4. add_messages + RemoveMessage 이해
- add_messages 사용시, RemoveMessage 값을 넘기면 지울수있음.

- 참고사항 1) 단순히 `RemoveMessage 사용시, id 값이 없으면 삭제할때 에러`발생 (id 값 없어서)

- 참고사항 2) 만약, `graph로 구성하고 RemoveMessage 사용시`에는 문제없음 (id 값 자동부여됨.)

RemoveMessage 사용

In [7]:
messages = [AIMessage("Hi.", 
                      name="Bot", 
                      id="1")]
messages.append(HumanMessage("Hi.", 
                             name="Lance", 
                             id="2"))
messages.append(AIMessage("So you said you were researching ocean mammals?", 
                          name="Bot", 
                          id="3"))
messages.append(HumanMessage("Yes, I know about whales. But what others should I learn about?", 
                             name="Lance", 
                             id="4"))

delete_messages = [RemoveMessage(id=m.id) for m in messages[:-2]]
add_messages(messages , delete_messages)

[AIMessage(content='So you said you were researching ocean mammals?', additional_kwargs={}, response_metadata={}, name='Bot', id='3'),
 HumanMessage(content='Yes, I know about whales. But what others should I learn about?', additional_kwargs={}, response_metadata={}, name='Lance', id='4')]

Graph + RemoveMessage 사용

In [14]:
messages = [AIMessage("Hi."),
            HumanMessage("Hi."),
            AIMessage("So you said you were researching ocean mammals?"),
            HumanMessage("Yes, I know about whales. But what others should I learn about?")]

@trace_function()
def filter_messages(state: MessagesState):
    for m in state["messages"][:-2]:
        print(m)
    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:-2]]
    return {"messages": delete_messages}

@trace_function()
def chat_model_node(state: MessagesState):    
    return {"messages": [llm.invoke(state["messages"])]}

builder = StateGraph(MessagesState)
builder.add_node("filter", filter_messages)
builder.add_node("chat_model", chat_model_node)
builder.add_edge(START, "filter")
builder.add_edge("filter", "chat_model")
builder.add_edge("chat_model", END)
graph = builder.compile()
graph.invoke({'messages': messages})


🚀 Passing Through [filter_messages] ..

#### [Input State]
  args: ({'messages': [AIMessage(content='Hi.', additional_kwargs={}, response_metadata={}, id='f6c7b12e-daf9-4b4e-9c6f-c4d9f7f0d4e1'), HumanMessage(content='Hi.', additional_kwargs={}, response_metadata={}, id='fe6bf41f-df7f-49c2-b8d0-19ed4d2f72d6'), AIMessage(content='So you said you were researching ocean mammals?', additional_kwargs={}, response_metadata={}, id='124545d4-4ec9-4036-9072-33a0a1ce1bc4'), HumanMessage(content='Yes, I know about whales. But what others should I learn about?', additional_kwargs={}, response_metadata={}, id='cedc66dd-8f7f-4ad5-9834-fb56401eafc9')]},)
  kwargs: {}
content='Hi.' additional_kwargs={} response_metadata={} id='f6c7b12e-daf9-4b4e-9c6f-c4d9f7f0d4e1'
content='Hi.' additional_kwargs={} response_metadata={} id='fe6bf41f-df7f-49c2-b8d0-19ed4d2f72d6'

#### [Output State]
  result: {'messages': [RemoveMessage(content='', additional_kwargs={}, response_metadata={}, id='f6c7b12e-daf9-4b4e-9c6f-

{'messages': [AIMessage(content='So you said you were researching ocean mammals?', additional_kwargs={}, response_metadata={}, id='124545d4-4ec9-4036-9072-33a0a1ce1bc4'),
  HumanMessage(content='Yes, I know about whales. But what others should I learn about?', additional_kwargs={}, response_metadata={}, id='cedc66dd-8f7f-4ad5-9834-fb56401eafc9'),
  AIMessage(content='In addition to whales, there are several other fascinating ocean mammals you might want to explore:\n\n1. **Dolphins**: Known for their intelligence and playful behavior, dolphins are highly social creatures. There are many species, including the bottlenose dolphin and the orca, or killer whale, which is actually the largest member of the dolphin family.\n\n2. **Porpoises**: Often confused with dolphins, porpoises are generally smaller and have different shaped teeth and dorsal fins. The harbor porpoise is one of the most well-known species.\n\n3. **Seals**: These pinnipeds are known for their sleek bodies and excellent sw

##### 5. trim messages 이해
- 전체 대화 텍스트에 대해 max_token 기준으로 자름.
- 아래 예시에서는 max_tokens=80으로 잘랐을경우, 최근 대화 2개만 남음.

In [26]:
test_messages = [HumanMessage(content="Hi, this is changwoo. How are you?"),
                 AIMessage(content="I'm fine, thank you. How can I help you?"),
                 HumanMessage(content="I'm looking for information on whales. What should I know about them?"),
                 AIMessage(content="Whales are fascinating creatures. They are the largest animals on Earth. They are found in all oceans, but they are most commonly found in the Pacific Ocean. They are known for their intelligence and their ability to communicate with each other.")]
messages = trim_messages(test_messages,
                         max_tokens=80,
                         strategy="last",
                         token_counter=ChatOpenAI(model="gpt-4o"),
                         allow_partial=False)
messages

[HumanMessage(content="I'm looking for information on whales. What should I know about them?", additional_kwargs={}, response_metadata={}),
 AIMessage(content='Whales are fascinating creatures. They are the largest animals on Earth. They are found in all oceans, but they are most commonly found in the Pacific Ocean. They are known for their intelligence and their ability to communicate with each other.', additional_kwargs={}, response_metadata={})]